# Leadgen Mortgage Final Model

In [1]:
import pandas as pd
import numpy as np

import pyodbc
# from fnbbase.connection import * 
# from pandasql import sqldf

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pysqldf = lambda q: sqldf(q, globals())
from sklearn import metrics

import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit
import scipy

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, GridSearchCV, StratifiedKFold, StratifiedShuffleSplit

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [2]:
# Data removed for privacy reasons
mtgDemo = pd.read_csv(, index_col = "hkey")
mtgDemoMinusOnly = pd.read_csv(, index_col = "hkey")
mtgDemo.head()

SyntaxError: invalid syntax (<ipython-input-2-adfb5d4e2510>, line 2)

## Logistic Regression Dummy Variables

Since there are more variables to keep than to exclude, we reversed engineered our variable assignment by dropping our categorical variables first. Then we dummied the categories out, and added them back into the model. We also excluded the dummy variables that contained NA or LEVEL U for espscore which is also NA.

In [ ]:
# Split data into explanatory and response variables
Y_all = mtgDemo["has_mtg"]
X_all = mtgDemo.drop(columns = [])
dummies1 = pd.get_dummies(mtgDemo[[]])
X_all = pd.concat([X_all, dummies1], axis = 1)
X_all = X_all.drop(columns = [])

Y_notOnly = mtgDemoMinusOnly["has_mtg"]
X_notOnly = mtgDemoMinusOnly.drop(columns = [])
dummies2 = pd.get_dummies(mtgDemoMinusOnly[[]])
X_notOnly = pd.concat([X_notOnly, dummies2], axis = 1)
X_notOnly = X_notOnly.drop(columns = [])

In [ ]:
mtgDemo["Total Non-mortgage Accounts"] = mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[] + mtgDemo[]
mtgDemoMinusOnly["Total Non-mortgage Accounts"] = mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[] + mtgDemoMinusOnly[]

## Standardization

We decided to also standardize the dummy variables here, shouldn't theoretically make any difference to the final model.

In [ ]:
# Standardize features
scaler = StandardScaler()

X_all_std = scaler.fit_transform(X_all)
X_notOnly_std = scaler.fit_transform(X_notOnly)

In [ ]:
X_all_std = pd.DataFrame(X_all_std, columns = X_all.columns)
X_notOnly_std = pd.DataFrame(X_notOnly_std, columns = X_notOnly.columns)

It turns out the standard scaler function resets the index to the row numbers. Here we fix it back to the original hkey index using Y

In [ ]:
# Reset the index for the X data
y_all_index = Y_all.index
X_all_std = X_all_std.set_index(y_all_index)

y_notOnly_index = Y_notOnly.index
X_notOnly_std = X_notOnly_std.set_index(y_notOnly_index)

In [ ]:
# Adding a column of 1 for the intercepts
X_all_std = sm.add_constant(X_all_std)
X_notOnly_std = sm.add_constant(X_notOnly_std)

## Train, Validate, Test Split

In [ ]:
# 80/20 train/test split then a 80/20 split on train to get train/validate

X_all_pretrain, X_all_test, Y_all_pretrain, Y_all_test = train_test_split(X_all_std, Y_all, test_size = 0.3, shuffle = True, random_state = 1)
X_all_train, X_all_validate, Y_all_train, Y_all_validate = train_test_split(X_all_pretrain, Y_all_pretrain, test_size = 0.3, shuffle = True, random_state = 1)

X_notOnly_pretrain, X_notOnly_test, Y_notOnly_pretrain, Y_notOnly_test = train_test_split(X_notOnly_std, Y_notOnly, test_size = 0.3, shuffle = True, random_state = 1)
X_notOnly_train, X_notOnly_validate, Y_notOnly_train, Y_notOnly_validate = train_test_split(X_notOnly_pretrain, Y_notOnly_pretrain, test_size = 0.3, shuffle = True, random_state = 1)

all_cols = X_all_std.columns
notOnly_cols = X_notOnly_std.columns

In [ ]:
# Checking to make sure that the hkey remained the index after splitting
print(X_all_pretrain.index)
print(X_notOnly_pretrain.index)

## Balancing Training Data

#### Undersampling

In [ ]:
# Use a random under sampling method to balance our data
us = RandomUnderSampler(random_state = 0)

In [3]:
us_data_X_all_train, us_data_Y_all_train = us.fit_resample(X_all_train, Y_all_train)
print(us_data_X_all_train.shape)
us_data_X_all_train = pd.DataFrame(data = us_data_X_all_train, columns = all_cols)
us_data_Y_all_train = pd.DataFrame(data = us_data_Y_all_train)
print(us_data_X_all_train.shape)


NameError: name 'us' is not defined

In [ ]:
print("Length of Undersampled Data:", len(us_data_X_all_train))
print("Number without a mortgage in Undersampled Data:", len(us_data_Y_all_train[us_data_Y_all_train["has_mtg"]==0]))
print("Number with a mortgage in Undersampled Data:", len(us_data_Y_all_train[us_data_Y_all_train["has_mtg"]==1]))
print("Proportion without a mortgage in Undersampled Data:", len(us_data_Y_all_train[us_data_Y_all_train["has_mtg"]==0])/len(us_data_X_all_train))
print("Proportion with a mortgage in Undersampled Data:", len(us_data_Y_all_train[us_data_Y_all_train["has_mtg"]==1])/len(us_data_X_all_train))

In [4]:
us_data_X_notOnly_train, us_data_Y_notOnly_train = us.fit_resample(X_notOnly_train, Y_notOnly_train)
us_data_X_notOnly_train = pd.DataFrame(data = us_data_X_notOnly_train, columns = notOnly_cols)
us_data_Y_notOnly_train = pd.DataFrame(data = us_data_Y_notOnly_train)

NameError: name 'us' is not defined

In [5]:
print("Length of Undersampled Data:", len(us_data_X_notOnly_train))
print("Number without a mortgage in Undersampled Data:", len(us_data_Y_notOnly_train[us_data_Y_notOnly_train["has_mtg"]==0]))
print("Number with a mortgage in Undersampled Data:", len(us_data_Y_notOnly_train[us_data_Y_notOnly_train["has_mtg"]==1]))
print("Proportion without a mortgage in Undersampled Data:", len(us_data_Y_notOnly_train[us_data_Y_notOnly_train["has_mtg"]==0])/len(us_data_X_notOnly_train))
print("Proportion with a mortgage in Undersampled Data:", len(us_data_Y_notOnly_train[us_data_Y_notOnly_train["has_mtg"]==1])/len(us_data_X_notOnly_train))

NameError: name 'us_data_X_notOnly_train' is not defined

## Model Validation

The following steps are performed 4 times
1. CV over train into train / val for 10 folds
- Run model on each train / val set
- Record Error Metrics
2. Select Best Model based on Error Metric
- Record Prediction Scores
3. Predict on test data and see if metrics are in the same range
4. Decile non-mortgage customers

#### Undersampling - All Mortgage Customers

In [6]:
# Step 1 - run model on each training and validation set
# Record error metric

sfk = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 123)
logisticRegr = LogisticRegression(fit_intercept = False, C = 1e9)

f1_scores = []
precision_scores = []
preds = []
models_all = []
logitmodels_all = []
sumlst_all = [] # Saving the summary list so we can get the pvalues

for test_ind,val_ind in sfk.split(us_data_X_all_train, us_data_Y_all_train):
    
    us_y_arr_all = us_data_Y_all_train.iloc[test_ind].to_numpy().ravel()
    
    # Sklearn model
    model = logisticRegr.fit(us_data_X_all_train.iloc[test_ind],us_y_arr_all)
    models_all.append(model)
    
    # Logit model
    result = sm.Logit(us_y_arr_all, (us_data_X_all_train.iloc[test_ind])).fit(disp = 0)
    sumlst_all.append(result.summary())
    logitmodels_all.append(result)
    
    pred = logisticRegr.predict(us_data_X_all_train.iloc[val_ind])
    preds.append(pred)
    
    f1_scores.append(metrics.f1_score(us_data_Y_all_train.iloc[val_ind],pred))
    precision_scores.append(metrics.precision_score(us_data_Y_all_train.iloc[val_ind],pred))

NameError: name 'us_data_X_all_train' is not defined

In [7]:
# Step 2 - Select the best model from error metric
lst = pd.DataFrame(np.column_stack([f1_scores,precision_scores]), columns = ['F1 Scores', 'Precision Scores'])
print(lst.idxmax()) # print the fold with the highest scores
best_cv_all = lst['Precision Scores'].idxmax()
best_cv_all

ValueError: attempt to get argmax of an empty sequence

In [8]:
# Saving the model results for the best cv in both model types (logit and sklearn)
# Used later for a comparison table
logit_all_pvalues = logitmodels_all[best_cv_all].pvalues
logit_all_se = logitmodels_all[best_cv_all].bse
logit_all_coef = logitmodels_all[best_cv_all].params
us_all_coef = models_all[best_cv_all].coef_

NameError: name 'best_cv_all' is not defined

In [9]:
# Calculating the error metrics for the best model

preds_all_val = models_all[best_cv_all].predict(X_all_validate)

accuracy_us_all = metrics.accuracy_score(Y_all_validate, preds_all_val)
precision_us_all = metrics.precision_score(Y_all_validate, preds_all_val)
f1_score_us_all = metrics.f1_score(Y_all_validate, preds_all_val)
auc_score_us_all = metrics.roc_auc_score(Y_all_validate, preds_all_val)
confusion_us_all = metrics.confusion_matrix(Y_all_validate, preds_all_val)
recall_us_all = metrics.recall_score(Y_all_validate, preds_all_val)

print("Accuracy: {}".format(round(accuracy_us_all, 3)))
print("Precision: {}".format(round(precision_us_all,3)))
print("F1 score: {}".format(round(f1_score_us_all,3)))
print("AUC score: {}".format(round(auc_score_us_all,3)))
print("Recall score: {}".format(round(recall_us_all,3)))
print("Confusion Matrix: \n{}".format(confusion_us_all))

NameError: name 'best_cv_all' is not defined

In [10]:
# Step 3 - fit the best model to the test set and calculate prediction metrics
pred_all_test = models_all[best_cv_all].predict(X_all_test)

accuracy_us_all = metrics.accuracy_score(Y_all_test, pred_all_test)
precision_us_all = metrics.precision_score(Y_all_test, pred_all_test)
f1_score_us_all = metrics.f1_score(Y_all_test, pred_all_test)
auc_score_us_all = metrics.roc_auc_score(Y_all_test, pred_all_test)
confusion_us_all = metrics.confusion_matrix(Y_all_test, pred_all_test)
recall_us_all = metrics.recall_score(Y_all_test, pred_all_test)

print("Accuracy: {}".format(round(accuracy_us_all, 3)))
print("Precision: {}".format(round(precision_us_all,3)))
print("F1 score: {}".format(round(f1_score_us_all,3)))
print("AUC score: {}".format(round(auc_score_us_all,3)))
print("Recall score: {}".format(round(recall_us_all,3)))
print("Confusion Matrix: \n{}".format(confusion_us_all))

NameError: name 'best_cv_all' is not defined

In [11]:
# Get the probabilities
us_all_proba = models_all[best_cv_all].predict_proba(X_all_test)

# Combine the probabilites with the original data so we can tie it back to the customer
X_all_test_reset = X_all_test.reset_index()
prob = pd.DataFrame(us_all_proba, columns = ['Neg', 'Pos'])
us_all_prob_list = pd.merge(prob,X_all_test_reset,how = 'inner', left_index = True, right_index = True)

# # Decile list
binx = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
us_all_leads = us_all_prob_list.Pos.value_counts(bins = binx).sort_index(ascending = False)
#print(us_all_leads)

NameError: name 'best_cv_all' is not defined

#### Undersampling - Cross Sell Mortgage Customers Only

In [12]:
# Step 1 - run model on each training and validation set
# Record error metric

sfk = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 123)
logisticRegr = LogisticRegression(fit_intercept = False, C = 1e9)

f1_scores = []
precision_scores = []
preds = []
models_notOnly = []
sumlst_notOnly = [] # Saving the summary list so we can get the pvalues
logitmodels_notOnly= []

for test_ind,val_ind in sfk.split(us_data_X_notOnly_train, us_data_Y_notOnly_train):
    
    us_y_arr_notOnly = us_data_Y_notOnly_train.iloc[test_ind].to_numpy().ravel()
    
    # Sklearn model
    model = logisticRegr.fit(us_data_X_notOnly_train.iloc[test_ind],us_y_arr_notOnly)
    models_notOnly.append(model)
    
    # Logit model
    result = sm.Logit(us_y_arr_notOnly, (us_data_X_notOnly_train.iloc[test_ind])).fit(disp = 0)
    sumlst_notOnly.append(result.summary())
    logitmodels_notOnly.append(result)
    
    pred = logisticRegr.predict(us_data_X_notOnly_train.iloc[val_ind])
    preds.append(pred)
    
    f1_scores.append(metrics.f1_score(us_data_Y_notOnly_train.iloc[val_ind],pred))
    precision_scores.append(metrics.precision_score(us_data_Y_notOnly_train.iloc[val_ind],pred))

NameError: name 'us_data_X_notOnly_train' is not defined

In [13]:
# Step 2 - Select the best model from error metric
pd.set_option('display.max_rows', 100)
lst = pd.DataFrame(np.column_stack([f1_scores,precision_scores]), columns = ['F1 Scores', 'Precision Scores'])
print(lst.idxmax())
best_cv_notOnly = lst['Precision Scores'].idxmax()
best_cv_notOnly

ValueError: attempt to get argmax of an empty sequence

In [14]:
# Saving the model results for the best cv in both model types (logit and sklearn)
# Used later for a comparison table
logit_notOnly_pvalues = logitmodels_notOnly[best_cv_notOnly].pvalues
logit_notOnly_se = logitmodels_notOnly[best_cv_notOnly].bse
logit_notOnly_coef = logitmodels_notOnly[best_cv_notOnly].params
us_notOnly_coef = models_notOnly[best_cv_notOnly].coef_

NameError: name 'best_cv_notOnly' is not defined

In [15]:
# Step 3 - fit the best model to the validation set and record error metrics

pred_notOnly_val = models_notOnly[best_cv_notOnly].predict(X_notOnly_validate)

accuracy_us_notOnly = metrics.accuracy_score(Y_notOnly_validate, pred_notOnly_val)
precision_us_notOnly = metrics.precision_score(Y_notOnly_validate, pred_notOnly_val)
f1_score_us_notOnly = metrics.f1_score(Y_notOnly_validate, pred_notOnly_val)
auc_score_us_notOnly = metrics.roc_auc_score(Y_notOnly_validate, pred_notOnly_val)
confusion_us_notOnly = metrics.confusion_matrix(Y_notOnly_validate, pred_notOnly_val)
recall_us_notOnly = metrics.recall_score(Y_notOnly_validate, pred_notOnly_val)

print("Accuracy: {}".format(round(accuracy_us_notOnly, 3)))
print("Precision: {}".format(round(precision_us_notOnly,3)))
print("F1 score: {}".format(round(f1_score_us_notOnly,3)))
print("AUC score: {}".format(round(auc_score_us_notOnly,3)))
print("Recall score: {}".format(round(recall_us_notOnly,3)))
print("Confusion Matrix: \n{}".format(confusion_us_notOnly))

NameError: name 'best_cv_notOnly' is not defined

In [16]:
# Step 3 - fit the best model to the test set and calculate prediction metrics
pred_notOnly_test = models_notOnly[best_cv_notOnly].predict(X_notOnly_test)

accuracy_us_notOnly = metrics.accuracy_score(Y_notOnly_test, pred_notOnly_test)
precision_us_notOnly = metrics.precision_score(Y_notOnly_test, pred_notOnly_test)
f1_score_us_notOnly = metrics.f1_score(Y_notOnly_test, pred_notOnly_test)
auc_score_us_notOnly = metrics.roc_auc_score(Y_notOnly_test, pred_notOnly_test)
confusion_us_notOnly = metrics.confusion_matrix(Y_notOnly_test, pred_notOnly_test)
recall_us_notOnly = metrics.recall_score(Y_notOnly_test, pred_notOnly_test)

print("Accuracy: {}".format(round(accuracy_us_notOnly, 3)))
print("Precision: {}".format(round(precision_us_notOnly,3)))
print("F1 score: {}".format(round(f1_score_us_notOnly,3)))
print("AUC score: {}".format(round(auc_score_us_notOnly,3)))
print("Recall score: {}".format(round(recall_us_notOnly,3)))
print("Confusion Matrix: \n{}".format(confusion_us_notOnly))

NameError: name 'best_cv_notOnly' is not defined

In [17]:
# Get the probabilities
us_notOnly_proba = models_notOnly[best_cv_notOnly].predict_proba(X_notOnly_test)

# Combine the probabilites with the original data so we can tie it back to the customer
# the predict_proba resets the index to the row numbers but keeps the same order so we need to normalize them
data_notOnly_test_reset = X_notOnly_test.reset_index()
prob = pd.DataFrame(us_notOnly_proba, columns = ['Neg', 'Pos'])
us_notOnly_prob_list = pd.merge(prob,data_notOnly_test_reset,how = 'inner', left_index = True, right_index = True)

# # Decile list
binx = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
us_notOnly_leads = us_notOnly_prob_list.Pos.value_counts(bins = binx).sort_index(ascending = False)
#print(us_notOnly_leads)

NameError: name 'best_cv_notOnly' is not defined

### Comparison Table

In [18]:
# Comparison table for the error metrics based on the test set
metric_table = pd.DataFrame(
    {'US All': [accuracy_us_all, precision_us_all, f1_score_us_all, auc_score_us_all, recall_us_all],
     'US Cross Sell': [accuracy_us_notOnly, precision_us_notOnly, f1_score_us_notOnly, auc_score_us_notOnly, recall_us_notOnly]},
     index = ['Accuracy', 'Precision','F1 Score','AUC Score','Recall'])

metric_table

NameError: name 'accuracy_us_all' is not defined

In [19]:
# Comparison table for deciles
leads_table = pd.DataFrame(
    {'US All': us_all_leads,
     'US Cross Sell': us_notOnly_leads})

leads_table

NameError: name 'us_all_leads' is not defined

In [20]:
# Make the coefficient comparison table
coefs_table = pd.DataFrame(
    {'All Coef': us_all_coef.ravel().round(3),
     'All Logit Coef': logit_all_coef.values.round(3),
     'All SE': logit_all_se,
     'All pval': logit_all_pvalues.round(4),
     'CS Coef': us_notOnly_coef.ravel().round(3),
     'CS Logit Coef': logit_notOnly_coef.values.round(3),
     'CS SE': logit_notOnly_se,
     'CS pval': logit_notOnly_pvalues.round(4)},
    index = X_all_std.columns)

# Add conditional column
coefs_table['CS Sig'] = (coefs_table['CS pval'] < 0.075)
coefs_table['All Sig'] =(coefs_table['All pval'] < 0.075)
coefs_table['Sign Match'] =(np.sign(coefs_table['CS Coef']) == np.sign(coefs_table['All Coef']))
coefs_table['Both Sig'] = ((coefs_table['All pval'] < 0.075) & (coefs_table['CS pval'] < 0.075))

print("# of Significant Variables in All: {}".format(coefs_table[(coefs_table["All Sig"] == True)].count()[0]))
print("# of Significant Variables in CS: {}".format(coefs_table[(coefs_table["CS Sig"] == True)].count()[0]))
print("# of Significant Variables in All AND CS: {}".format(coefs_table[(coefs_table["Both Sig"] == True)].count()[0]))

# Table that shows all variables where the significance differs between the two models
coefs_table[(coefs_table["All Sig"] != coefs_table['CS Sig'])]

NameError: name 'us_all_coef' is not defined

In [21]:
# Table that shows all variables where it was significant in at least one model but the coefficient signs don't match
coefs_table[((coefs_table["Sign Match"] == False) & ((coefs_table["All Sig"] == True) | (coefs_table["CS Sig"] == True )))]

NameError: name 'coefs_table' is not defined

## Decile and Lift

#### Decile Function from Bill

In [22]:
# Decile function from Bill that calculates deciles and lift
def decile_df(preds, actuals, y_train):
    deciles = pd.qcut(preds,q=10,labels=False)
    preds_df = pd.DataFrame(preds.values, index=actuals.index, columns=['preds'])
    preds_df['actuals'] = actuals.values
    preds_df['decile'] = deciles
    dec_df = preds_df.groupby('decile').agg(['min','max','mean','count'])
    dec_df.index = range(10,0,-1)
    dec_df.columns = ['_'.join(col).strip() for col in dec_df.columns.values]
    dec_df2 = dec_df[['preds_min','preds_max','actuals_mean','preds_count']]
    dec_df2.columns = ['min','max','avg','n']
    baseline = y_train.mean() # actuals.mean()#[0]
    dec_df2['lift'] = dec_df2['avg']/baseline
    return dec_df2.sort_index(), preds_df

### Cross Sell Mortgage Customers

In [23]:
# Get the list of probabilities 
probs_notOnly = us_notOnly_prob_list.set_index('hkey')['Pos']

# Get the actual values in the test set
actuals_notOnly = Y_notOnly_test

NameError: name 'us_notOnly_prob_list' is not defined

In [24]:
# Checking the shape of the parameters for the decile function
print(probs_notOnly.shape,actuals_notOnly.shape,Y_notOnly_train.shape)

NameError: name 'probs_notOnly' is not defined

In [25]:
# Calculate the deciles and lift
lr_dec_notOnly, lr_dec_by_idx_notOnly = decile_df(probs_notOnly, actuals_notOnly, Y_notOnly_pretrain)
lr_dec_notOnly

NameError: name 'probs_notOnly' is not defined

### All Mortgage Customers

In [26]:
# Get the list of probabilities 
probs_all = us_all_prob_list.set_index('hkey')['Pos']
# Get the actual values in the test set
actuals_all = Y_all_test

NameError: name 'us_all_prob_list' is not defined

In [27]:
# Checking the shape of the parameters for the decile function
print(probs_all.shape,actuals_all.shape,Y_all_pretrain.shape)

NameError: name 'probs_all' is not defined

In [28]:
# Calculate the deciles and lift
lr_dec_all, lr_dec_by_idx_all = decile_df(probs_all, actuals_all, Y_all_pretrain)
lr_dec_all

NameError: name 'probs_all' is not defined

In [29]:
mtgDemoWithPreds = pd.merge(lr_dec_by_idx_all,mtgDemo,how = 'inner',left_index = True, right_index = True)
mtgDemoMinusOnlyWithPreds = pd.merge(lr_dec_by_idx_notOnly,mtgDemoMinusOnly,how = 'inner',left_index = True, right_index = True)
mtgDemoWithPreds["Binary preds"] = mtgDemoWithPreds["preds"].round(0)
mtgDemoMinusOnlyWithPreds["Binary preds"] = mtgDemoMinusOnlyWithPreds["preds"].round(0)
#print(mtgDemoWithPreds[mtgDemoWithPreds["Total Non-mortgage Accounts"] == 1]["Binary preds"].value_counts())
#print(mtgDemoWithPreds[mtgDemoWithPreds["Total Non-mortgage Accounts"] == 1]["actuals"].value_counts())
print(mtgDemoMinusOnlyWithPreds[mtgDemoMinusOnlyWithPreds["Total Non-mortgage Accounts"] == 1]["Binary preds"].value_counts())
print(mtgDemoMinusOnlyWithPreds[mtgDemoMinusOnlyWithPreds["Total Non-mortgage Accounts"] == 1]["actuals"].value_counts())
#print(mtgDemoWithPreds[mtgDemoWithPreds["Total Non-mortgage Accounts"] > 1]["Binary preds"].value_counts())
#print(mtgDemoWithPreds[mtgDemoWithPreds["Total Non-mortgage Accounts"] > 1]["actuals"].value_counts())
print(mtgDemoMinusOnlyWithPreds[mtgDemoMinusOnlyWithPreds["Total Non-mortgage Accounts"] > 1]["Binary preds"].value_counts())
print(mtgDemoMinusOnlyWithPreds[mtgDemoMinusOnlyWithPreds["Total Non-mortgage Accounts"] > 1]["actuals"].value_counts())

NameError: name 'lr_dec_by_idx_all' is not defined

## ABC Table

In [30]:
# Diagonals
# A leads
A_leads_all = lr_dec_by_idx_all[lr_dec_by_idx_all.decile == 9]
A_leads_notOnly = lr_dec_by_idx_notOnly[lr_dec_by_idx_notOnly.decile == 9]
A_leads_both = pd.merge(A_leads_all,A_leads_notOnly,how = 'inner',left_index = True, right_index = True)

# B leads
B_leads_all = lr_dec_by_idx_all[lr_dec_by_idx_all.decile == 8]
B_leads_notOnly = lr_dec_by_idx_notOnly[lr_dec_by_idx_notOnly.decile == 8]
B_leads_both = pd.merge(B_leads_all,B_leads_notOnly,how = 'inner',left_index = True, right_index = True)

# C leads
C_leads_all = lr_dec_by_idx_all[lr_dec_by_idx_all.decile == 7]
C_leads_notOnly = lr_dec_by_idx_notOnly[lr_dec_by_idx_notOnly.decile == 7]
C_leads_both = pd.merge(C_leads_all,C_leads_notOnly,how = 'inner',left_index = True, right_index = True)

# No leads
no_leads_all = lr_dec_by_idx_all[(lr_dec_by_idx_all.decile != 7) & (lr_dec_by_idx_all.decile != 8) & (lr_dec_by_idx_all.decile != 9)]
no_leads_notOnly = lr_dec_by_idx_notOnly[(lr_dec_by_idx_notOnly.decile != 7) & (lr_dec_by_idx_notOnly.decile != 8) & (lr_dec_by_idx_notOnly.decile != 9)]
no_leads_both = pd.merge(no_leads_all,no_leads_notOnly,how = 'inner',left_index = True, right_index = True)

# Off Diagonals
A_all_B_notOnly = pd.merge(A_leads_all,B_leads_notOnly,how = 'inner',left_index = True, right_index = True)
B_all_A_notOnly = pd.merge(B_leads_all,A_leads_notOnly,how = 'inner',left_index = True, right_index = True)
A_all_C_notOnly = pd.merge(A_leads_all,C_leads_notOnly,how = 'inner',left_index = True, right_index = True)
C_all_A_notOnly = pd.merge(C_leads_all,A_leads_notOnly,how = 'inner',left_index = True, right_index = True)
B_all_C_notOnly = pd.merge(B_leads_all,C_leads_notOnly,how = 'inner',left_index = True, right_index = True)
C_all_B_notOnly = pd.merge(C_leads_all,B_leads_notOnly,how = 'inner',left_index = True, right_index = True)
no_all_A_notOnly = pd.merge(no_leads_all,A_leads_notOnly,how = 'inner',left_index = True, right_index = True)
no_all_B_notOnly = pd.merge(no_leads_all,B_leads_notOnly,how = 'inner',left_index = True, right_index = True)
no_all_C_notOnly = pd.merge(no_leads_all,C_leads_notOnly,how = 'inner',left_index = True, right_index = True)
A_all_no_notOnly = pd.merge(A_leads_all,no_leads_notOnly,how = 'inner',left_index = True, right_index = True)
B_all_no_notOnly = pd.merge(B_leads_all,no_leads_notOnly,how = 'inner',left_index = True, right_index = True)
C_all_no_notOnly = pd.merge(C_leads_all,no_leads_notOnly,how = 'inner',left_index = True, right_index = True)

pd.DataFrame({'Cross Sell A': [A_leads_both.count()[0],B_all_A_notOnly.count()[0],C_all_A_notOnly.count()[0],no_all_A_notOnly.count()[0]],
             'Cross Sell B': [A_all_B_notOnly.count()[0],B_leads_both.count()[0],C_all_B_notOnly.count()[0],no_all_B_notOnly.count()[0]],
             'Cross Sell C': [A_all_C_notOnly.count()[0],B_all_C_notOnly.count()[0],C_leads_both.count()[0],no_all_C_notOnly.count()[0]],
             'Cross Sell DQ': [A_all_no_notOnly.count()[0],B_all_no_notOnly.count()[0],C_all_no_notOnly.count()[0],no_leads_both.count()[0]]},
             index = ['All A','All B','All C','All DQ'])

NameError: name 'lr_dec_by_idx_all' is not defined

In [31]:
pd.DataFrame({'Cross Sell A': [A_leads_both.sum()[1],B_all_A_notOnly.sum()[1],C_all_A_notOnly.sum()[1],no_all_A_notOnly.sum()[1]],
             'Cross Sell B': [A_all_B_notOnly.sum()[1],B_leads_both.sum()[1],C_all_B_notOnly.sum()[1],no_all_B_notOnly.sum()[1]],
             'Cross Sell C': [A_all_C_notOnly.sum()[1],B_all_C_notOnly.sum()[1],C_leads_both.sum()[1],no_all_C_notOnly.sum()[1]],
             'Cross Sell DQ': [A_all_no_notOnly.sum()[1],B_all_no_notOnly.sum()[1],C_all_no_notOnly.sum()[1],no_leads_both.sum()[1]]},
             index = ['All A','All B','All C','All DQ'])

NameError: name 'A_leads_both' is not defined

## Off Diagonal Confusion Matrices

In [32]:
# not the most beautiful thing in the world but it gets the job done
print("All B, Cross Sell A \n{}".format(metrics.confusion_matrix(B_all_A_notOnly['actuals_x'],B_all_A_notOnly['preds_x'].round(0))))
print("All C, Cross Sell B \n{}".format(metrics.confusion_matrix(C_all_B_notOnly['actuals_y'],C_all_B_notOnly['preds_y'].round(0))))
print("All A, Cross Sell C \n{}".format(metrics.confusion_matrix(A_all_C_notOnly['actuals_y'],A_all_C_notOnly['preds_y'].round(0))))
print("All C, Cross Sell A \n{}".format(metrics.confusion_matrix(C_all_A_notOnly['actuals_y'],C_all_A_notOnly['preds_y'].round(0))))
print("All DQ, Cross Sell A \n{}".format(metrics.confusion_matrix(no_all_A_notOnly['actuals_y'],no_all_A_notOnly['preds_y'].round(0))))
print("All DQ, Cross Sell B \n{}".format(metrics.confusion_matrix(no_all_B_notOnly['actuals_y'],no_all_B_notOnly['preds_y'].round(0))))
print("All DQ, Cross Sell C \n{}".format(metrics.confusion_matrix(no_all_C_notOnly['actuals_y'],no_all_C_notOnly['preds_y'].round(0))))
print("All A, Cross Sell B \n{}".format(metrics.confusion_matrix(A_all_B_notOnly['actuals_y'],A_all_B_notOnly['preds_y'].round(0))))
print("All B, Cross Sell C \n{}".format(metrics.confusion_matrix(B_all_C_notOnly['actuals_y'],B_all_C_notOnly['preds_y'].round(0))))
print("All A, Cross Sell DQ \n{}".format(metrics.confusion_matrix(A_all_no_notOnly['actuals_y'],A_all_no_notOnly['preds_y'].round(0))))
print("All B, Cross Sell DQ \n{}".format(metrics.confusion_matrix(B_all_no_notOnly['actuals_y'],B_all_no_notOnly['preds_y'].round(0))))
print("All C, Cross Sell DQ \n{}".format(metrics.confusion_matrix(C_all_no_notOnly['actuals_y'],C_all_no_notOnly['preds_y'].round(0))))

NameError: name 'B_all_A_notOnly' is not defined